In [3]:
import numpy as np
import cupy as cp
import cupyx.scipy.signal as signal
import scipy
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from astropy.modeling import models, fitting
import astropy.units as u
from astropy import constants as const        
from astropy.stats import gaussian_sigma_to_fwhm,gaussian_fwhm_to_sigma

import datetime
import pickle
import importlib
import json
import time
import glob

from time_converter import time_converter
# from data.eve_const import wavelength_full,line_name,line_window

In [18]:
with open('data/selected_band.pkl','rb') as f:
    selected_band = pickle.load(f)

In [19]:
selected_band

,Line Name,Wavelength Range,Initial Guess,Wavelength Index
0,X148,"[14.75, 14.93]","[0.00011, 14.84, 0.0424]","[588, 589, 590, 591, 592, 593, 594, 595, 596]"
1,X150,"[14.93, 15.09]","[4.4e-05, 15.01, 0.0424]","[597, 598, 599, 600, 601, 602, 603, 604]"
2,X152,"[15.13, 15.35]","[6.5e-05, 15.22, 0.0424]","[607, 608, 609, 610, 611, 612, 613, 614, 615, ..."
3,X154,"[15.35, 15.53]","[4e-05, 15.42, 0.0424]","[618, 619, 620, 621, 622, 623, 624, 625, 626]"
4,Fe171,"[17, 17.21]","[0.0006, 17.11, 0.0424]","[700, 701, 702, 703, 704, 705, 706, 707, 708, ..."
5,X174,"[17.37, 17.59]","[0.00069, 17.45, 0.0424]","[719, 720, 721, 722, 723, 724, 725, 726, 727, ..."
6,X176,"[17.63, 17.83]","[0.00045, 17.72, 0.0424]","[732, 733, 734, 735, 736, 737, 738, 739, 740, ..."
7,Fe195,"[17.95, 18.15]","[0.0006, 19.51, 0.0424]","[748, 749, 750, 751, 752, 753, 754, 755, 756, ..."
8,X182,"[18.15, 18.31]","[0.00013, 18.22, 0.0424]","[758, 759, 760, 761, 762, 763, 764, 765]"
9,X185,"[18.35, 18.59]","[0.0002, 18.45, 0.0424]","[768, 769, 770, 771, 772, 773, 774, 775, 776, ..."


In [2]:
gaussian_fwhm_to_sigma

0.42466090014400953

In [4]:
eve_full_files=sorted(glob.glob('./data/EVE_sav_full/EVS_L2*.sav'))
len(eve_full_files)

36099

# A first look at MEGS-A and MEGS-B data form 10 to 103 nm

In [4]:
eve_data=scipy.io.readsav(eve_full_files[400])
eve_data.keys()

dict_keys(['wavelength', 'sod_time', 'irradiance', 'sc_flags', 'flags', 'yyyydoy'])

: 

In [7]:
# plt.plot(eve_data['wavelength'])
print(np.min(eve_data['wavelength']),np.max(eve_data['wavelength']))
# This version includes all measured wavelengths spanning 5.8 - 106.2 nm

3.01 106.99


In [5]:
eve_data=scipy.io.readsav(eve_full_files[26354])
wavelength_full

# Extracting the relevant data
a=eve_data['irradiance'][0]
t=time_converter(eve_data['yyyydoy'][0],eve_data['sod_time'][0])


# Creating a figure using Plotly Graph Objects
fig = go.Figure(data=go.Scatter(x=wavelength_full[:],y=a[:], mode='lines'))

fig.update_yaxes(#title_text="Stddev (nm)",
              range=[0, 0.006],
                )
fig.update_layout(title_text="EVE Irradiance"+'<br>'+t.strftime('%Y-%m-%d %H:%M:%S'),
                  xaxis_title="Wavelength (nm)",
                  yaxis_title="Irradiance (W/m^2/nm)",
                  )
# fig.write_html('output/full_band_spectrum/overview1.html')

In [26]:
eve_data1=scipy.io.readsav(eve_full_files[400])
wavelength_full=eve_data1['wavelength']
# Extracting the relevant data
a1=eve_data1['irradiance'][0]
t1=time_converter(eve_data1['yyyydoy'][0],eve_data1['sod_time'][0])


eve_data2=scipy.io.readsav(eve_full_files[11500])
# Extracting the relevant data
a2=eve_data2['irradiance'][0]
t2=time_converter(eve_data2['yyyydoy'][0],eve_data2['sod_time'][0])

eve_data3=scipy.io.readsav(eve_full_files[26354])
# Extracting the relevant data
a3=eve_data3['irradiance'][0]
t3=time_converter(eve_data3['yyyydoy'][0],eve_data3['sod_time'][0])


# Creating a figure using Plotly Graph Objects
fig = go.Figure()
# draw a line purple with opacity=0.5
fig.add_trace(go.Scatter(x=wavelength_full[:],y=a1,
                          mode='lines',
                          line=dict(color='green'),
                          opacity=0.5,
                            name=t1.strftime('%Y-%m-%d %H:%M:%S')
                            ),  
)

fig.add_trace(go.Scatter(x=wavelength_full[:],y=a2, 
                         mode='lines',
                         line=dict(color='lightblue'),
                          # opacity=0.5,
                          name=t2.strftime('%Y-%m-%d %H:%M:%S')
                          ),
),
fig.add_trace(go.Scatter(x=wavelength_full[:],y=a3,
                          mode='lines',
                          line=dict(color='mediumblue'),
                          opacity=0.5,
                            name=t3.strftime('%Y-%m-%d %H:%M:%S')
                            ),
  )



# for index in line_window:

#   fig.add_shape(type="rect",
#               x0=wavelength_full[index[0]], y0=0,
#               x1=wavelength_full[index[-1]], y1=0.006,
#               # set line color to opaque
#               line_color="orangered",
#               fillcolor="orangered",
#               opacity=0.5,
#               layer="below"
#               )

for index in selected_band.index:
  if selected_band.loc[index]['Line Name'][0]=='X':
    fig.add_shape(type="rect",
              x0=selected_band.loc[index]['Wavelength Range'][0], y0=0,
              x1=selected_band.loc[index]['Wavelength Range'][-1], y1=0.006,
              # set line color to opaque
              line_color="lightgreen",
              fillcolor="lightgreen",
              opacity=0.5,
              layer="below"
              )
  else:
    fig.add_shape(type="rect",
              x0=selected_band.loc[index]['Wavelength Range'][0], y0=0,
              x1=selected_band.loc[index]['Wavelength Range'][-1], y1=0.006,
              # set line color to opaque
              line_color="orangered",
              fillcolor="orangered",
              opacity=0.5,
              layer="below"
              )


fig.update_yaxes(#title_text="Stddev (nm)",
              range=[0, 0.006],
                )
fig.update_layout(title_text="EVE Irradiance",
                  xaxis_title="Wavelength (nm)",
                  yaxis_title="Irradiance (W/m^2/nm)",
                  )
fig.write_html('output/full_band_spectrum/overview.html')
fig.show()

In [17]:
selected_band.loc[index]

Line Name                               X174
Wavelength Range              [17.37, 15.59]
Initial Guess       [0.00069, 17.45, 0.0424]
Wavelength Index                          []
Name: 5, dtype: object

In [ ]:
# draw vertical line at 20 and 30,then add a purple with opaque=0.5 to the space in between
fig.add_shape(type="line",
    x0=20, y0=0, x1=20, y1=0.006,
    line=dict(color="purple",width=1,dash="dot")
)
fig.add_shape(type="line",
    x0=30, y0=0, x1=30, y1=0.006,
    line=dict(color="purple",width=1,dash="dot")
)
fig.add_shape(type="rect",
    x0=20, y0=0, x1=30, y1=0.006,
    fillcolor="purple",
    opacity=0.5,
    layer="below"
)

In [6]:
gaussian_sigma_to_fwhm

2.3548200450309493

In [54]:
wavelength_list_full[index[0]]

array([103.05, 103.07, 103.09, 103.11, 103.13, 103.15, 103.17, 103.19,
       103.21, 103.23, 103.25, 103.27, 103.29, 103.31, 103.33],
      dtype='>f4')

In [58]:
line_window[0]

(array([700, 701, 702, 703, 704, 705, 706, 707, 708, 709], dtype=int64),)

In [60]:
np.where((wavelength_list_full<17.2)*(wavelength_list_full>17))[0]

array([700, 701, 702, 703, 704, 705, 706, 707, 708, 709], dtype=int64)

# Select Lines

FWHM=0.1nm

sigma=FWHM/2.355=0.0424nm

window=0.20nm    corresponding to [-2.3sigma,2.3sigma]

windows=0.30nm   corresponding to [-3.4sigma,3.4sigma]

In [2]:
# AIA lines 但是其中的很多不明显没法高斯拟合
lines=['Fe94', 'Fe131', 'Fe171', 'Fe193', 'Fe211', 'He304']
line_centers = [9.4, 13.1, 17.1, 19.3, 21.1, 30.4]


In [7]:
# Fe93 absolutely unapplicable
# Fe131 unapplicable
# Fe171
index1=np.where((wavelength_full<=17.21)*(wavelength_full>17))[0]
#Fe180
index2=np.where((wavelength_full<=18.15)*(wavelength_full>=17.95))[0]
# Fe195 
index3=np.where((wavelength_full<=19.59)*(wavelength_full>=19.43))[0]
# Fe284
index4=np.where((wavelength_full<=28.53)*(wavelength_full>=28.31))[0]
# He 304
index5=np.where((wavelength_full<=30.49)*(wavelength_full>=30.25))[0]
# Fe335 absolutely unapplicable


# O 6297
index6=np.where((wavelength_full<=63.09)*(wavelength_full>=62.85))[0]
# O 9770
index7=np.where((wavelength_full<=97.85)*(wavelength_full>=97.57))[0]
# O 1032
index8=np.where((wavelength_full<=103.35)*(wavelength_full>=103.05))[0]




# Constants
line_name=['Fe171', 'Fe180', 'Fe195', 'Fe284', 'He304','O630', 'O977', 'O1032']
line_window=[index1,index2,index3,index4,index5,index6,index7,index8]

In [12]:
# save wavelength_full
np.savez('./data/wavelength_full.npz',wavelength_full=wavelength_full)